# 卷积神经网络

在使用卷积之前，人们尝试了很多人工神经网络来处理图像问题，但是人工神经网络的参数量非常大，从而导致训练难度非常大。因此，计算机视觉的研究一直停滞不前，难以突破。

直到卷积神经网络的出现，它的两个优秀特点：*稀疏连接*与*平移不变性*，让计算机视觉的研究取得了长足的进步。稀疏连接可以让学习的参数变得很少，而平移不变性则不关心物体出现在图像中什么位置。

## 卷积的计算

### 最简单的卷积

卷积核是什么？如下图中红色的示例，其中的卷积核是最简单的情况，只有一个通道。

步长是什么？卷积上下左右滑动的长度，称为步长，用 stride 表示。根据问题的不同，会取不同的步长，但通常来说步长为 1 或 2。

如下，输入是一个 4x4 的特征图，卷积核的大小为 2x2。

输入特征与卷积核计算时，计算方式是卷积核与输入特征按位做乘积运算然后再求和，其结果为输出特征图的一个元素，下图为计算输出特征图第一个元素的计算方式：

<img src="https://static001.geekbang.org/resource/image/78/20/787c8b346de00dayyd7e2d3504c33320.jpg?wh=1561x891" width="40%" />

完成了第一个元素的计算，接着按从左向右，从上至下的顺序进行滑动卷积核，分别与输入的特征图进行计算，向右侧滑动一个单元的计算方式：

<img src="https://static001.geekbang.org/resource/image/5y/b5/5yy249d2f1221e21a1bdc7d8756f4fb5.jpg?wh=1544x862" width="40%" />

。。。
第一行计算完毕之后，卷积核会回到行首，然后向下滑动一个单元，再重复以上从左至右的滑动计算。

<img src="https://static001.geekbang.org/resource/image/cf/bb/cf4aa3yy8ac31b06f153f9090d3bcebb.jpg?wh=1552x929" width="40%" />


### 标准的卷积

现在将最简单的卷积计算方式延伸到标准的卷积计算方式。

假定，输入的特征有 m 个通道，宽为 w，高为 h；输出有 n 个特征图，宽为 w′，高为 h′；卷积核的大小为 k * k。

这个操作的卷积是什么样子的？
1. 输出特征图的通道数由卷积核的个数决定 ==>卷积核的个数 n。
2. 根据卷积计算的定义，输入特征图有 m 个通道 ==>每个卷积核里要也要有 m 个通道。

所以，需要 n 个卷积核，每个卷积核的大小为 (m, k, k)。

<img src="https://static001.geekbang.org/resource/image/62/12/62fd6c269cee17e778f8d5acf085be12.jpeg?wh=1920x1080" width="50%" />

结合上面的图解可以看到，
- 卷积核 1 与全部输入特征进行卷积计算，就获得了输出特征图中第 1 个通道的数据
- 卷积核 2 与全部输入特征图进行计算获得输出特征图中第 2 个通道的数据。
- 以此类推，最终就能计算 n 个输出特征图。
- 现在有 m 个通道，输入特征的每一个通道与卷积核中对应通道的数据按上面的方式进行卷积计算，也就是输入特征图中第 i 个特征图与卷积核中的第 i 个通道的数据进行卷积。这样计算后会生成 m 个特征图，然后将这 m 个特征图按对应位置求和即可，求和后 m 个特征图合并为输出特征中一个通道的特征图。

用公式表示当输入有多个通道时，每个卷积核是如何与输入进行计算的：
- Outputi​ 表示计算第 i 个输出特征图，i 的取值为 1 到 n；
- kernelk​ 表示 1 个卷积核里的第 k 个通道的数据；
- inputk​ 表示输入特征图中的第 k 个通道的数据；
- biasi​ 为偏移项，我们在训练时一般都会默认加上；
- 卷积计算：
   <img src="../res/images/convolution-calc.jpg" width="40%" />

   >为什么要加 bias。与回归方程相似，如果不加 bias 的话，回归方程为 y=wx 不管 w 如何变化，回归方程都必须经过原点。如果加上 bias 的话，回归方程变为 y=wx+b，这样就不是必须经过原点，可以变化的更加多样。

## padding

有的时候可能需要对特征图进行补零操作，这样做主要有两个目的：
1. 需要输入与输出的特征图保持一样的大小；
2. 让输入的特征保留更多的信息。

一般什么情况下会希望特征图变得不那么小？

如果不补零且步长（stride）为 1 的情况下，当有多层卷积层时，特征图会一点点变小。如果希望有更多层卷积层来提取更加丰富的信息时，就可以考虑补零让特征图变小的速度稍微慢一些。这个补零的操作就叫做 padding，padding 等于 1 就是补一圈的零，等于 2 就是补两圈的零。

<img src="https://static001.geekbang.org/resource/image/99/08/99dc22a96df665e93e881yy3cf358d08.jpg?wh=1520x792" width="40%" />

在 Pytorch 中，padding 这个参数可以是字符串、int 和 tuple。
现在分别来看看不同参数类型的使用：
- 当为字符串时只能取 `valid` 与 `same`。
- 当给定整型时，则是要在特征图外边补多少圈 0。
- 给定 tuple 时，则表示在特征图的行与列分别指定补多少零。

**padding 设置为 `same`**

再来看上面卷积计算的例子，

当滑动到特征图最右侧时，发现输出的特征图的宽与输入的特征图的宽不一致，它会自动补零，直到输出特征图的宽与输入特征图的宽一致为止。如下图所示：

<img src="https://static001.geekbang.org/resource/image/52/c7/52f8d0ba39b49e9a2736c1a0afb38cc7.jpg?wh=1463x720" width="40%" />

同理，当计算到特征图的底部时，发现输出特征图的高与输入特征图的高不一致时，它同样会自动补零，直到输入和输出一致为止，如下图所示：

<img src="https://static001.geekbang.org/resource/image/c4/81/c48614da6c7bbcd5abdaf942ea45b481.jpg?wh=1476x735" width="40%" />

完成上述操作，就可以获得与输入特征图有相同高、宽的输出特征图了。

## PyTorch 中的卷积

卷积操作定义在 torch.nn 模块中，这个模块提供了很多构建网络的基础层与方法，如， nn.Conv1d、nn.Conv2d 与 nn.Conv3d 。
nn.Conv2d 使用得最多，而 nn.Conv1d 与 nn.Conv3d 只是输入特征图的维度有所不一样而已，很少会被用到。

```python
class torch.nn.Conv2d(in_channels, 
                      out_channels, 
                      kernel_size, 
                      stride=1, 
                      padding=0, 
                      dilation=1, 
                      groups=1, 
                      bias=True, 
                      padding_mode='zeros', 
                      device=None, 
                      dtype=None)
```

1. in_channels 是指输入特征图的通道数，数据类型为 int。如上标准卷积中 in_channels 为 m；
1. out_channels 是输出特征图的通道数，数据类型为 int。如上标准卷积中 out_channels 为 n。
1. kernel_size 是卷积核的大小，数据类型为 int 或 tuple，需要注意的是只给定卷积核的高与宽即可。如上标准卷积中 kernel_size 为 k。
1. stride 为滑动的步长，数据类型为 int 或 tuple，默认是 1，在前面的例子中步长都为 1。
1. padding 为补零的方式，注意当 padding 为`valid` 与 `same`时，stride 必须为 1。
1. bias 是否使用偏移项。
1. dilation
1. groups

>对于 kernel_size、stride、padding 都可以是 tuple 类型，当为 tuple 类型时，第一个维度用于 height 的信息，第二个维度时用于 width 的信息。
>
>卷积背后的理论比较复杂，但在 PyTorch 中实现却很简单。在卷积计算中涉及的几大要素：输入通道数、输出通道数、步长、padding、卷积核的大小，分别对应的就是 nn.Conv2d 的关键参数。所以，要熟练用好 nn.Conv2d()。


### 验证 same 方式

In [1]:
import torch
import torch.nn as nn

创建（4，4，1）大小的输入特征图

In [2]:
input_feat = torch.tensor([[4, 1, 7, 5], [4, 4, 2, 5], [7, 7, 2, 4], [1, 0, 2, 4]], dtype=torch.float32)
print(input_feat)
print(input_feat.shape)

tensor([[4., 1., 7., 5.],
        [4., 4., 2., 5.],
        [7., 7., 2., 4.],
        [1., 0., 2., 4.]])
torch.Size([4, 4])


创建一个 2x2 的卷积

In [3]:
conv2d = nn.Conv2d(1, 1, (2, 2), stride=1, padding='same', bias=True)

print(conv2d.weight)
print(conv2d.bias)

Parameter containing:
tensor([[[[-0.2843,  0.1378],
          [ 0.4674,  0.1638]]]], requires_grad=True)
Parameter containing:
tensor([0.0220], requires_grad=True)


强行干预卷积核的初始化

*默认情况下是随机初始化的*，不会人工强行干预卷积核的初始化，但是为了验证 `padding=same`，我们对卷积核的参数进行干预。代码如下：

In [4]:
conv2d = nn.Conv2d(1, 1, (2, 2), stride=1, padding='same', bias=False)

# 卷积核要有四个维度(输入通道数，输出通道数，高，宽)
kernels = torch.tensor([[[[1, 0], [2, 1]]]], dtype=torch.float32)

conv2d.weight = nn.Parameter(kernels, requires_grad=False)
print(conv2d.weight)
print(conv2d.bias)

Parameter containing:
tensor([[[[1., 0.],
          [2., 1.]]]])
None


完成以上两部，就可以开始计算了，使用已经准备好的输入数据与卷积数据计算得到输出，代码如下：

In [ ]:
output = conv2d(input_feat)

# RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [4, 4]

这里报错了，提示信息是输入的特征图需要是一个 4 维的，而给定的输入特征图是一个 4x4 的 2 维特征图。这是为什么呢？

Pytorch 输入 tensor 的维度信息是 (batch_size, 通道数，高，宽)，但上面只给定了高与宽，没有给定 batch_size（在训练时，不会将所有数据一次性加载进来训练，而是以多个批次进行读取的，每次读取的量成为 batch_size）与通道数。所以，回到第一步将输入的 tensor 改为 (1,1,4,4) 的形式。

In [6]:
# input_feat = torch.tensor([[4, 1, 7, 5], [4, 4, 2, 5], [7, 7, 2, 4], [1, 0, 2, 4]], dtype=torch.float32)
input_feat = input_feat.unsqueeze(0).unsqueeze(0)
print(input_feat)
print(input_feat.shape)


tensor([[[[4., 1., 7., 5.],
          [4., 4., 2., 5.],
          [7., 7., 2., 4.],
          [1., 0., 2., 4.]]]])
torch.Size([1, 1, 4, 4])


In [8]:
output = conv2d(input_feat)

output

tensor([[[[16., 11., 16., 15.],
          [25., 20., 10., 13.],
          [ 9.,  9., 10., 12.],
          [ 1.,  0.,  2.,  4.]]]])